In [1]:
import sys

# Insert utils folder into path
sys.path.insert(1, '../utils')

## Training / Fine-tuning Process

In [2]:
task = "ner"
model_checkpoint = "bert-base-multilingual-cased" # mBERT pre-trained from HuggingFace Hub
batch_size = 16

### Loading the dataset

In [3]:
from datasets import load_dataset, load_metric, concatenate_datasets

datasets = load_dataset("conll2003")

all_data = concatenate_datasets([datasets['train'], datasets['validation'], datasets['test']])

Reusing dataset conll2003 (C:\Users\Bernard\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


In [4]:
label_list = all_data.features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

### Processing the data

In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
tokenized_dataset = all_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/21 [00:00<?, ?ba/s]

In [8]:
from sklearn.model_selection import KFold

In [9]:
n = 5
seed = 40
kf = KFold(n_splits=n, random_state=seed, shuffle=True)

In [10]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        "LOC-f1": results['LOC']["f1"],
        "LOC-precision": results['LOC']["precision"],
        "LOC-recall": results['LOC']["recall"],
    }

In [11]:
from cross_validation_mBERT import cross_validation_mBERT

In [12]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import EarlyStoppingCallback

fold = 1

args = TrainingArguments(
    output_dir=f"fold-{fold}-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

metric = load_metric("seqeval")
data_collator = DataCollatorForTokenClassification(tokenizer)

for train_index, eval_index in kf.split(tokenized_dataset):
    
    train_data = tokenized_dataset.select(train_index)
    eval_data = tokenized_dataset.select(eval_index)
    
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
    
    trainer = Trainer(model,
                      args,
                      train_dataset=train_data,
                      eval_dataset=eval_data,
                      data_collator=data_collator,
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics)
    
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
    
    trainer.train()
    
    cross_validation_mBERT(model, tokenizer, label_list, fold)
    
    fold += 1


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Loc-f1,Loc-precision,Loc-recall,Runtime,Samples Per Second
1,0.084400,0.084179,0.899461,0.921194,0.910198,0.979094,0.941332,0.936762,0.945946,6.369400,651.397000
2,0.051500,0.079931,0.922732,0.939195,0.930891,0.982944,0.958111,0.964804,0.951510,6.284800,660.163000
3,0.031600,0.071331,0.937112,0.943035,0.940065,0.985363,0.958131,0.949285,0.967144,6.358600,652.506000
4,0.022300,0.076697,0.935810,0.941275,0.938535,0.985030,0.962490,0.966346,0.958665,6.362500,652.105000
5,0.015000,0.086339,0.934252,0.943595,0.938901,0.985092,0.955003,0.953993,0.956015,6.399800,648.299000
6,0.010200,0.087011,0.936207,0.945196,0.940680,0.985178,0.956774,0.951757,0.961844,6.286100,660.023000
7,0.007900,0.091336,0.937554,0.946556,0.942034,0.985709,0.960649,0.963981,0.957340,6.374300,650.891000
8,0.003600,0.091812,0.937406,0.947756,0.942553,0.985635,0.959616,0.965406,0.953895,6.421500,646.107000
9,0.003000,0.098552,0.938524,0.949036,0.943750,0.986067,0.962234,0.965830,0.958665,6.467600,641.509000
10,0.001800,0.097202,0.940241,0.950396,0.945291,0.986449,0.960404,0.963220,0.957605,6.298800,658.699000


  0%|          | 0/2 [00:00<?, ?ba/s]

Evaluation mode: strict
fp: 1523 | tp: 3872 | fn: 1671
precision: 0.718 | recall: 0.699 | f-score: 0.708 | accuracy: 0.699
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 238 | tp: 5157 | fn: 992
precision: 0.956 | recall: 0.839 | f-score: 0.893 | accuracy: 0.930
------------------------------------------------------------------------



  0%|          | 0/1 [00:00<?, ?ba/s]

...

Evaluation mode: strict
fp: 288 | tp: 935 | fn: 383
precision: 0.765 | recall: 0.709 | f-score: 0.736 | accuracy: 0.709
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 82 | tp: 1141 | fn: 295
precision: 0.933 | recall: 0.795 | f-score: 0.858 | accuracy: 0.866
------------------------------------------------------------------------



  0%|          | 0/1 [00:00<?, ?ba/s]

Evaluation mode: strict
fp: 1604 | tp: 3174 | fn: 1914
precision: 0.664 | recall: 0.624 | f-score: 0.643 | accuracy: 0.624
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 723 | tp: 4055 | fn: 1430
precision: 0.849 | recall: 0.739 | f-score: 0.790 | accuracy: 0.797
------------------------------------------------------------------------



  0%|          | 0/1 [00:00<?, ?ba/s]

Evaluation mode: strict
fp: 486 | tp: 1526 | fn: 1072
precision: 0.758 | recall: 0.587 | f-score: 0.662 | accuracy: 0.587
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 92 | tp: 1920 | fn: 918
precision: 0.954 | recall: 0.677 | f-score: 0.792 | accuracy: 0.739
------------------------------------------------------------------------

